Copyright 2022 DeepMind Technologies Limited. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2022 DeepMind Technologies Limited
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction

This notebook introduces the code framework for reproducing the results of the NeurIPS paper,

Malek, Alan, and Silvia Chiappa. "Asymptotically Best Causal Effect Identification with Multi-Armed ceb.bandits." Advances in Neural Information Processing Systems 34 (2021).

We will introduce the code stucture by way of a few examples before providing the code that generated plots in the paper.

## Implementation details 
We begin by going through the various classes and what abstractions they reperesent. The main package is `causal_effect_bandits`, which we will import as `ceb` throughout.

### Passing Data
Observational data is stored in the `ceb.data.ExpData` class, which keeps numpy arrays for covariates $Z$, exposure $X$, and response $Y$. For data `d`, they can be accessed by `d.cov`, `d.exp`, and `d.rsp`, respectively. Throughout this notebook, we will always use $Z$ for covariates, $X$ for exposure, and $Y$ for response.

### Generating Observational Data
The `data` module also has a `ceb.data.DataGenerator` class with a `.generate(n)` method that will produce an `ceb.data.ExpData` object of `n` samples. The most common data generating process we will use is a Structural Causal Model (`ceb.data.SCM`), which is implemented in the `SCM` module.

### Modelling an Estimator
Our estimators are specified by an uncentered influence function $\phi$ and a nuisance parameter $\eta$; these two objects are implemented by the `ceb.parameters.NuisanceParameter` class, where each instance has the following components:
- object `self._model`, representing $\eta$, that we may `.fit` and `.transform` (with the usual sklearn AIP)
- method `fit(W: ceb.data.ExpData)` that uses observation data $W$ to fit $\hat\eta(W)$
- method `_phi(eta: np.ndarray,  W: ceb.data.ExpData)` that  return a vector $(\phi(w_i, \eta_i): \eta_i \in \text{eta}, w_i \in \text{W})$
- method `calculate_score(W: ceb.data.ExpData)` function that first calculates $\eta$ from `W then returns the array $(\phi(w, \hat\eta(w)): w \in \text{W})$, on the fit $\hat\eta$.
- method `calculate_cs_width(delta, args*)`: calculates a confidence interval width on the error of the nuissanc parameter, $\Vert \hat\eta - \eta\Vert$. This error is needed by the confidence intervals for the asymptotic variance estimator.

Specific examples of eta (e.g. the AIPW) will be implemented by subclassing `ceb.parameters.NuissanceParameter`, including the following classes
  - `ceb.parameters.LinearParameter`: fits the two conditional response function, $\mu(1, z)$ and $\mu(0, z)$, using linear regression
  -`ceb.parameters.AIPW`: fits the augmented inverse propensity weight estimator by fitting the conditional response $\mathbb E[Y|Z,X]$ and the propensity score $e(z) = p(X=1|Z=z)$.
  - `ceb.parameters.FrontdoorLinearFiniteX`: implements the frontdoor formula described in the paper appendix.


### Estimating the Asymptotic Variance
The `ceb.arms.VarianceEstimatorArm` represents an estimator of the causal effect $\tau_k$, and contains code to build a confidence sequence on an estimate of the asymptotic variance of the estimator $\sigma_k^2$. This class has the following attributes and methods.
* object `self._eta`, a `parameter.NuisanceParameter` instance.
* method `update(new_data: ceb.data.ExpData, delta: float)`: updates the confidence sequences and parameter estimates using data `new_data` such that the total error of the confidence sequence is at most `delta`.
* method `reset()`: resets the arms to its initial value, including the parameter estimate in `self._eta`.
* properties `ci`, `ate`, and `var_est`, which hold the current confidence interval, estimate of $\tau_k$, and estimate of $\sigma^2_k$, respectively.
* attribute `self._data_transformer: Callable[[ceb.data.ExpData], ceb.data.ExpData]`, a function. This takes an `ceb.data.ExpData` object of observational data (e.g. generated by a `ceb.data.DataGenerator` object) and transforms it into an `ceb.data.ExpData` object that is expected by `self._eta`. Examples of these functions are returned by 
 * `ceb.data.get_identity_fn()` (which returns the identity function),
 * `ceb.data.get_remove_coordinate_fn(idx)` which returns a function that maps an `ceb.data.ExpData` object into an `ceb.data.ExpData` object with the same `.exp` and `.rsp` and the `.cov` with the coordinates in idx removed.
 * `ceb.data.get_coordinate_mask_fn(idx)` which returns a function that maps an `ceb.data.ExpData` object into an `ceb.data.ExpData` object with the same `.exp` and `.rsp` and the `.cov` with only the coordinates in idx remaining.


The most common example is if an `ceb.data.ExpData` object `d` contains more covariates in `d.cov` than is needed by the `VarianceEstimatorArm`. Suppose `d` has `d.cov` with two dimensions, the first corresponding to $Z_1$ and the second corresponding to $Z_2$. Then creating an `arm.VarianceEstimatorArm` with `_data_transformer = ceb.data.get_remove_coordinate_fn(1)` corresponds to an arm that only sees $Z_1$, and creating an `arm.VarianceEstimatorArm` with `_data_transformer = ceb.data.get_remove_coordinate_fn(0)` corresponds to an arm that only sees $Z_2$.

### Running a Bandit Algorithm over Arms
A `ceb.bandits.BanditAlgorithm` class holds a collection of `ceb.arms.VarianceEstimatorArm` and a `ceb.data.DataGenerator` and implements a specific bandit algorithm (which specifies which arm to pull during the next round) and generates data from the `ceb.data.DataGenerator` object. Important attributes and methods for this class include the following. 

The only important method is `self.run(all_data: Optional[ceb.data.ExpData])`, which runs the bandit algorithm either on the provided dataset, or on the `ceb.data.DataGenerator` provided. Specifying a static dataset makes comparing the performances of various bandit algorithms have lower variance. This method returns a `ceb.bandits.BanditData`object, which is a `dataclass` that contains:
* `lcb_by_arm`  a dictionary with `arm.VarianceEstimatorArm` keys that collects an array of all the the lower confidence bound of that arm for every iteration of the bandit algorithm
* `ucb_by_arm` an analogous dictionary of arrays for the upper confidence bounds.
* `var_est_by_arm` an analogous dictionary of arrays for the variance estimates.
* `samples_by_arm` an analogous dictionary of arrays for the number of new samples used (it is not cumulative).
* `cum_samples`: an array of the cumulative number of samples used by every arm for every iteration of the bandit algorithm
* `best_arms`: a list of the best arm(s).

Bandit algorithms implemented include:
* `UniformAlgorithm`: samples every arm every round
* `LUCBAlgorithm`: implements the LUCB algorithm from
``` Kalyanakrishnan, Shivaram, et al. "PAC subset selection in stochastic multi-armed ceb.bandits." ICML. Vol. 12. 2012.```

* 'SuccessiveEliminationAlgorithm`: implements the successive elimination algorithm from 
```Even-Dar, Eyal, et al. "Action Elimination and Stopping Conditions for the Multi-Armed Bandit and Reinforcement Learning Problems." Journal of machine learning research 7.6 (2006).```

## Setup and Importing Packages

In [ ]:
!pip install git+https://github.com/deepmind/abcei_mab

In [ ]:
# Imports
import causal_effect_bandits as ceb
import numpy as np
import matplotlib.pyplot as plt

## Minimum Working Example
With all that out of the way, let us start with a simple SCM that has three nodes, Z, X, and Y. The edges in this graph are $X\leftarrow Z\rightarrow Y$ and $X\rightarrow Y$; that is, $Z$ confounds $X$ and $Y$ so 
we cannot get an unbiased estimate for the treatment effect if we ignore $Z$.

We also include an irrelevent variable $Z_2$, statistically independent from the other variables (i.e. with no parents or children) to 
illustrate how to use data_transformer to ignore it.

We now specify the conditional probability distributions of each node
by building a dictionary. We will define
- $Z \sim$ Bernoulli$(1/3)$ (it has no parents), 
- $T|Z=z \sim$ Bernoulli$(1/2 + .1z)$,
- $Y|X=x,Z=z \sim$ Normal$(1/2 + x/4 - z/4, 0)$, and
- $Z_2 \sim$ Normal$(0)$

Each cpd is a function mapping from n (a sample size) and parents, a 
List[np.ndarray], where each entry in the list is of size(n, parent_dimension), 
and the length of the list is equal to the number of parents of the node.


In [ ]:
var_names = ["Z", "Z2", "X", "Y"]
cov_variables = ["Z", "Z2"]
treatment_variable = "X"
response_variable = "Y"

# We specify the graph by listing the parents of each node.
parents = {
    "Z": [],
    "X": ["Z"],
    "Y": ["Z2", "X"],
    "Z2": ["Z"],
}

cpds = {
    "Z":
        lambda n, parents: np.random.binomial(1, 1/3, size=n),
    "X":
        lambda n, parents: np.random.binomial(1, .5 + .1 * parents[0]),
    "Z2":
        lambda n, parents: np.random.normal(.1 * parents[0], 1),
    "Y":      
        lambda n, parents: np.random.normal(size=len(parents[0])) + .25 *
          parents[1] + .25 * parents[0],
}

# We can now call the SCM constructor and generate data
scm_gen = ceb.scm.SCM(
    name="back-door example",
    var_names=var_names,
    parents=parents,
    cpds=cpds,
    cov_variables=cov_variables,
    treatment_variable=treatment_variable,
    response_variable=response_variable,
)

# Finally, we generate 10 datapoints and print the ExpData object.
# The first column of Z corresponds to Z, which is binary, and the second 
# column to Z2, which is continuous.

scm_gen.generate(10)

_Expected output_:
A 2-dimensional covariates matrix, with the first collumn being binary, along with an exposure and response vectors, all of length 10.

The next step is to define two VarianceEstimatorArms using the back-door formula a.k.a. covariate adjustment. One arm will just use $Z$ and one will use $Z$ and $Z_2$. 

We need to specify the scale of the problem, which we do by estimating the variance on a small dataset.

In [ ]:
d = scm_gen.generate(1000)
remove_Z2 = ceb.data.get_remove_coordinates_fn(1)
remove_Z = ceb.data.get_remove_coordinates_fn(0)
Z_var = np.var(remove_Z2(d).cov)
ZZ2_var = np.var(d.cov)

armZ = ceb.arms.SampleSplittingArm(
      'Z_adjustment',
      eta=ceb.parameters.AIPWLinearLogistic(n_features=1),
      # remove the Z2 coordinate
      data_transformer=remove_Z2,
      ci_algorithm=ceb.arms.CIAlgorithm.CLT,
      sub_gscale=Z_var,
      tau_bound=2,
      burn_in=100,
      rho=1,
)
armZZ2 = ceb.arms.SampleSplittingArm(
      'Z2_adjustment',
      eta=ceb.parameters.AIPWLinearLogistic(n_features=1),
      # do not remove the Z2 coordinate
      data_transformer=remove_Z,
      ci_algorithm=ceb.arms.CIAlgorithm.CLT,
      sub_gscale=ZZ2_var,
      tau_bound=2,
      burn_in=100,
      rho=1,
)

# Next, we define the bandit instance. 
bandit = ceb.bandits.UniformAlgorithm(
    arm_list=[armZ, armZZ2],
    prob_model=scm_gen,
    confidence=.05,
    sample_limit=150000,
    units_per_round=1000,
)

# We run the bandit algorithm to get bandit data
np.random.seed(0)
bdata = bandit.run()
print(f"The best arm was found to be {bdata.best_arm[0].name}")

# and plot the results
fig, ax = plt.subplots(figsize=(20,10))
ceb.plotting_utils.plot_bandit_results(ax, [armZ, armZZ2], bdata, 1)
plt.show()



_Expected Output_:
We should see the confidence sequencences gradually tighten until the blue interval, which corresponds to adjusting by $Z$, stops intersecting the orange interval. At this point, the algorithm terminates and outputs that adjusting by $Z_2$ is superior, which agrees with the theory that predicts that adjusting by variables closer to the response $Y$ results in a lower variance estimator.

# Plots from the Paper

Next, we turn towards generating the plots from the paper. 

## Experiments from Section 5.1

The larger the number of possible adjustment sets, the bigger the potential sample savings a bandit algortihm can provide.

We will generalize the back door and front door example from the section. Instead of there being two back door paths $X\leftarrow V_i \rightarrow Z_i\rightarrow Y$, for $i=1,2$, we will consider $K$ back-door paths. Any valid adjustment set will have to block all of them, and so it must include either $V_i$ or $Z_i$ for all $i=1,\ldots, K$; therefore, there are $2^K$ valid adjustment sets.

In addition, we have a possible frontdoor adjustment path from $X\rightarrow Z_M\rightarrow Y$.

On a side note, we can include adjustment by the front and back door by using the expression derived in example 10 of Smucler and Rotnitzky, 2019. 

In [ ]:
# First, we define the numer of back-door paths and the dimension
num_back_door_paths = 3
sample_limit = 200000
units_per_round = 500
z_dim = [2] + [3 for _ in range(num_back_door_paths)]  # include Z0
v_dim = [0] + [2 for _ in range(num_back_door_paths)]  # include space for V0

# We generate the SCM, VarianceEstimatorArms, and the bandit algorithms
args = ceb.paper_utils.get_section_5_1_example(z_dim, 
                               v_dim,
                               seed=2,
                               z_cost=3,
                               v_cost=1,
                               z0_cost=3,
                               num_back_door_paths=3,
                               sample_limit = sample_limit,
                               units_per_round = units_per_round,
                               latex_names = True,
)
(scm_gen, arm_list, LUCB_bandit, SE_bandit, uniform_bandit) = args

# To help the comparison between the arms, we fix the the data
#all_data = scm_gen.generate(sample_limit + units_per_round * len(arm_list))

results = SE_bandit.run()
print(f"The best arm was found to be {results.best_arm[0].name} after {max(results.cum_samples)} samples were used.")

# Plot the results of the best m arms
m = 4
fig, ax = plt.subplots(figsize=(20,10))
plt.ticklabel_format(axis='x', style='scientific',scilimits=(0,0))
cutoff_var = np.sort([results.var_est_by_arm[a][-1] for a in arm_list])[m]
top_arms = [a for a in arm_list if results.var_est_by_arm[a][-1] < cutoff_var]
ceb.plotting_utils.plot_bandit_results(ax, top_arms, results, initial_points_to_ignore=4)
plt.show()

_Expected Output_: the frontdoor arm, in blue, should be the best.

## Section 5.1, continued

Next, we generate Figure 1, b), which requires that we run the three bandit algorithms on multiple instances and collect the sample complexities.

There is a lot of simulation, so this cell can take around an hour to finish running.

In [ ]:
n_instances = 10
n_repetitions = 5
num_back_door_paths = 3
sample_limit = 200000
units_per_round = 1000
Z_dim = [2] + [3 for _ in range(num_back_door_paths)]  # include Z0
V_dim = [0] + [2 for _ in range(num_back_door_paths)]  # include space for V0

LUCB_samples = []
SE_samples = []
uniform_bandit_samples = []
instance_history = []

instance = -1
while len(LUCB_samples) < n_instances * n_repetitions:
  instance += 1
  print(f"At instance {instance}")
  # Approximate the variances to make sure the instance isn't too hard.
  # We generate the SCM, VarianceEstimatorArms, and the bandit algorithms
  args = ceb.paper_utils.get_section_5_1_example(
      Z_dim,
      V_dim,
      seed=instance,
      z_cost=2,
      v_cost=1,
      z0_cost=5,
      num_back_door_paths=3,
      sample_limit=sample_limit,
      units_per_round=units_per_round,
  )
  (scm_gen, arm_list, LUCB_bandit, SE_bandit, uniform_bandit) = args

  temp_data = scm_gen.generate(10000)
  var_hat_tau = []
  for arm in arm_list:
    arm.reset()
    arm_data = arm._data_transformer(temp_data).k_fold(2)
    arm._eta.fit(arm_data[0])
    var_hat_tau.append(arm.cost * np.var(arm._eta.calculate_score(arm_data[1])))

  sorted_vars = np.array(var_hat_tau)
  sorted_vars.sort()
  min_gap = sorted_vars[1] - sorted_vars[0]

  if min_gap > 5:
    print(f"The minimum gap is {min_gap}. Proceeding!")
  else:
    print(f"The minimum gap is {min_gap}. Generating a new instance!")
    continue

  for m in range(n_repetitions):
    instance_history.append(instance)
    all_data = scm_gen.generate(sample_limit + units_per_round * len(arm_list))

    # Run LUCB
    LUCB_bandit.reset()
    np.random.seed(m)
    LUCB_results = LUCB_bandit.run(all_data)
    LUCB_samples.append(LUCB_results.cum_samples[-1])

    # Run SuccessiveElimination
    SE_bandit.reset()
    np.random.seed(m)
    SE_results = SE_bandit.run(all_data)
    SE_samples.append(SE_results.cum_samples[-1])
    
    # Calculate the number of samples needed by uniform_bandit
    total_samples_by_arm = [sum(SE_results.samples_by_arm[a]) for a in arm_list]
    max_samples = max(total_samples_by_arm)
    # uniform_bandit would use max_samples for every arm
    uniform_bandit_samples.append(max_samples * len(arm_list))
    
    print((f'instance {instance} had samples: \tLUCB:{LUCB_samples[-1]}\t'),
      (f'SE:{SE_samples[-1]}\tuniform:{uniform_bandit_samples[-1]}'))
  
fig, ax = plt.subplots()
plt.boxplot([LUCB_samples, SE_samples, uniform_bandit_samples],
            labels=['$CS-LUCB$', '$CS-SE$', '$Uniform$'])
plt.ticklabel_format(axis='y', style='scientific',scilimits=(0,0))
plt.ylabel("Samples")
plt.show()

_Expected Output_: The scrips iterates through instances (random seeds) until it finds a SCM with a large enough gap between the two ceb.arms. Then, it prints the number of samples needed by LUCB, SE, and the uniform algorithm for 5 different random datasets from this instance. The script them moves onto to a different random SCM. After all this is finished (which may take some time), we should see a box plot indicating that both LUCB and SE have substantially smaller sample complexities than the uniform algorithm, with SE being slightly better than LUCB.

## Experiments from Section 5.2

The causal graph is the same as the previous section, except all the functional relationships are nonlinear and sampled from a Gaussian Process. 

There is a lot of simulation, so expect potentially a few hours for this cell to finish running.

In [ ]:
n_instances = 10
n_repetitions = 5
num_back_door_paths = 3
sample_limit = 500000
units_per_round = 5000
Z_dim = [2] + [3 for _ in range(num_back_door_paths)]  # include Z0
V_dim = [0] + [2 for _ in range(num_back_door_paths)]  # include space for V0

LUCB_samples = []
SE_samples = []
uniform_bandit_samples = []
instance_history = []

instance = -1
while len(LUCB_samples) < n_instances * n_repetitions:
  instance += 1
  print(f"At instance {instance}")
  # Approximate the variances to make sure the instance isn't too hard.
  # We generate the SCM, VarianceEstimatorArms, and the bandit algorithms
  args = ceb.paper_utils.get_section_5_2_example(
      Z_dim,
      V_dim,
      seed=instance,
      z_cost=2,
      v_cost=1,
      z0_cost=5,
      num_back_door_paths=3,
      sample_limit=sample_limit,
      units_per_round=units_per_round,
  )
  (scm_gen, arm_list, LUCB_bandit, SE_bandit, uniform_bandit) = args

  temp_data = scm_gen.generate(10000)
  var_hat_tau = []
  for arm in arm_list:
    arm.reset()
    arm_data = arm._data_transformer(temp_data).k_fold(2)
    arm._eta.fit(arm_data[0])
    var_hat_tau.append(arm.cost * np.var(arm._eta.calculate_score(arm_data[1])))

  sorted_vars = np.array(var_hat_tau)
  sorted_vars.sort()
  min_gap = sorted_vars[1] - sorted_vars[0]

  if min_gap > 5:
    print(f"The minimum gap is {min_gap}. Proceeding!")
  else:
    print(f"The minimum gap is {min_gap}. Generating a new instance!")
    continue

  for m in range(n_repetitions):
    instance_history.append(instance)
    all_data = scm_gen.generate(sample_limit + units_per_round * len(arm_list))

    # Run LUCB
    LUCB_bandit.reset()
    np.random.seed(m)
    LUCB_results = LUCB_bandit.run(all_data)
    LUCB_samples.append(LUCB_results.cum_samples[-1])

    # Run SuccessiveElimination
    SE_bandit.reset()
    np.random.seed(m)
    SE_results = SE_bandit.run(all_data)
    SE_samples.append(SE_results.cum_samples[-1])
    
    # Calculate the number of samples needed by uniform_bandit
    total_samples_by_arm = [sum(SE_results.samples_by_arm[a]) for a in arm_list]
    max_samples = max(total_samples_by_arm)
    # uniform_bandit would use max_samples for every arm
    uniform_bandit_samples.append(max_samples * len(arm_list))

    print((f'instance {instance} had samples: \tLUCB:{LUCB_samples[-1]}\t'),
        (f'SE:{SE_samples[-1]}\tuniform:{uniform_bandit_samples[-1]}'))
  
fig, ax = plt.subplots()#figsize=(7,5))
plt.boxplot([LUCB_samples, SE_samples, u_bandit_samples], labels=['CS-LUCB','CS-SE','Uniform'])
plt.ticklabel_format(axis='y', style='scientific',scilimits=(0,0))
plt.ylim(0, 1.4e5)
plt.ylabel("Samples")
plt.show()

_Expected Output_: The output for this cell should look very similar to the output for Section 5.1, continued. The main difference is that the SCM is non-linear, and non-linear nuisance functions are used. The same level of sample complexity reduction is also present.

## Section 5.2, continued

This cell runs the algorithms for a varying number of ceb.arms. To control the number of arms, we vary $M$, the number of back-door paths; recall that we have $2^M+1$ arms ($2^M$ back-door and 1 frontdoor). For each $M$, run the algorithms over several different instances, then plot the average sample complexity.

This cell may take an hour or two to finish running.

In [ ]:
from collections import defaultdict

# Each element in back_door_size_list is run once
back_door_size_list = [3] * 10 + [4] * 10 + [5] * 5 + [6] * 5
sample_limit_list = [100000]*20 + [600000]*10

LUCB_samples = defaultdict(list)
SE_samples = defaultdict(list)
uniform_bandit_samples = defaultdict(list)
instance_history = []

units_per_round = 500

instance = -1
for (num_back_door, sample_limit) in zip(back_door_size_list, sample_limit_list):
  instance +=1
  Z_dim = [2] + [3 for _ in range(num_back_door)]  # include Z0
  V_dim = [0] + [2 for _ in range(num_back_door)]  # include space for V0
  print(f"At instance {instance} with {num_back_door} back-doors")
  # Approximate the variances to make sure the instance isn't too hard.    
  args = ceb.paper_utils.get_section_5_2_example(
      Z_dim,
      V_dim,
      seed=instance,
      z_cost=2,
      v_cost=1,
      z0_cost=5,
      num_back_door_paths=3,
      sample_limit=sample_limit,
      units_per_round=units_per_round,
  )

  (scm_gen, arm_list, LUCB_bandit, SE_bandit, uniform_bandit) = args
  instance_history.append(instance)

  np.random.seed(instance)
  all_data = scm_gen.generate(sample_limit + units_per_round * len(arm_list))
  
  np.random.seed(instance)
  LUCB_bandit.reset()
  LUCB_results = LUCB_bandit.run(all_data)
  LUCB_samples[num_back_door].append(LUCB_results.cum_samples[-1])

  np.random.seed(instance)
  SE_bandit.reset()
  SE_results = SE_bandit.run(all_data)
  SE_samples[num_back_door].append(SE_results.cum_samples[-1])
  
  total_samples = max([sum(SE_results.samples_by_arm[a]) for a in arm_list]) * len(arm_list)
  uniform_bandit_samples[num_back_door].append(total_samples)
        
  print((f'instance {instance} with M={num_back_door} had samples: \tLUCB:{LUCB_results.cum_samples[-1]}\t'),
       (f'SE:{SE_results.cum_samples[-1]}\tuniform:{total_samples}'))
  
# box plot code
back_door_size = np.array(list(set(back_door_size_list)))
num_arms = 1 + 2 ** back_door_size
SE_mean_samples = [np.mean(SE_samples[size]) for size in back_door_size]
LUCB_mean_samples = [np.mean(LUCB_samples[size]) for size in back_door_size]
uniform_mean_samples = [np.mean(uniform_bandit_samples[size]) for size in back_door_size]

fig, ax = plt.subplots(figsize=(7,5))
plt.plot(num_arms, SE_mean_samples, label='CS-SE')
plt.plot(num_arms, LUCB_mean_samples, label='CS-LUCB')
plt.plot(num_arms, uniform_mean_samples, label='Uniform')
plt.ylabel("Samples")
plt.rc('font', size=20)
plt.legend()
plt.show()


_Expected Results_: We should see the sample complexity of the uniform algorithm increase about linearly, while the sample complexity of LUCB and SE should increase much more slowly. This is because the sample complexity scales roughly with the sum of the reciprocal squared gaps, which tends to grow much slower than the number of ceb.arms. See Theorem 3 in the paper.